In [187]:
# !pip3 install beautifulsoup4

In [188]:
# !pip3 install requests

In [189]:
# !pip3 install pandas

In [190]:
# !pip3 install selenium

In [229]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By

# driver = webdriver.Chrome()
# for url in df['product_manual_url']:
#     driver.get(url)
#     driver.switch_to.new_window()

In [230]:
# driver.quit()

In [167]:
import os
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [168]:
with open('product_ids.txt','r') as file:
    lines = file.readlines()
    url_pattern = lines[1]
    current_wearables = lines[2]
    product_ids = current_wearables.split(',')

print(url_pattern)
print(product_ids)

https://www.garmin.com/en-US/p/{product_id}

['886725', '714945', '783467', '819761', '785411', '662790', '719696', '775697', '854515', '730659', '741137', '777730', '780139', '873214', '1057989', '894067', '866191', '886689', '641121', '847706', '818387', '782585', '713363', '582444', '698632', '643382', '646690', '1297277', '711488', '766565', '869433', '798926', '647267', '743387', '665374', '701618', '721216', '707225']


In [169]:
url = 'https://www.garmin.com/en-US/siteindex/'
response = requests.get(url)
sitemap_links = []

soup = BeautifulSoup(response.content, 'html.parser')
for link in soup.find_all('a'):
    href = link.get('href')
    if '/c/' in href:
        sitemap_links.append(href)

for link in sitemap_links[:3]:
    print(link)

https://www.garmin.com/en-US/c/wearables-smartwatches/
https://www.garmin.com/en-US/c/women-wearables/?FILTER_FEATURE_FORWOMEN=true
https://www.garmin.com/en-US/c/sports-fitness/fashion-hybrid-smartwatches/


In [165]:
def get_product_info(product_id):
    url = f'https://www.garmin.com/en-US/p/{product_id}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    scripts = soup.find_all('script')
    for script in scripts:
        if 'tagManagement_product_id' in script.text:
            product_category = re.search(r'var tagManagement_pageCategory = "(.*?)";', script.text).group(1)
            product_subcategory = re.search(r'var tagManagement_categoryName = "(.*?)";', script.text).group(1)
            product_name = re.search(r'var tagManagement_productName = "(.*?)";', script.text).group(1)
            product_id = re.search(r'var tagManagement_product_id = "(.*?)";', script.text).group(1)
            product_sku = re.search(r'var tagManagement_skuCode = "(.*?)";', script.text).group(1)
        
            product_dict = {
                'product_url': url,
                'product_id': product_id,
                'product_name': product_name,
                'product_category': product_category,
                'product_subcategory': product_subcategory,
                'part_number': product_sku,
                'product_manual_url': f'https://support.garmin.com/en-US/?partNumber={product_sku}&tab=manuals'
            }

            return product_dict
        else:
            continue

In [170]:
get_product_info(product_ids[0])

{'product_url': 'https://www.garmin.com/en-US/p/886725',
 'product_id': '886725',
 'product_name': 'Forerunner 965',
 'product_category': 'Sports & Fitness',
 'product_subcategory': 'Running',
 'part_number': '010-02809-00',
 'product_manual_url': 'https://support.garmin.com/en-US/?partNumber=010-02809-00&tab=manuals'}

In [172]:
product_info_list = []
df = pd.DataFrame(columns=['product_url','product_id','product_name',
                           'product_category','product_subcategory',
                           'part_number','product_manual_url'])
for id in product_ids:
    product_info = get_product_info(id)
    temp = pd.DataFrame(product_info, index=[0])
    df = pd.concat([df, temp], ignore_index=True)
    product_info_list.append(product_info)


In [177]:
df.head()

,product_url,product_id,product_name,product_category,product_subcategory,part_number,product_manual_url
0,https://www.garmin.com/en-US/p/886725,886725,Forerunner 965,Sports & Fitness,Running,010-02809-00,https://support.garmin.com/en-US/?partNumber=0...
1,https://www.garmin.com/en-US/p/714945,714945,Bounce,Sports & Fitness,Just for Kids,010-02448-02,https://support.garmin.com/en-US/?partNumber=0...
2,https://www.garmin.com/en-US/p/783467,783467,MARQ Adventurer (Gen 2),Sports & Fitness,Luxury Smartwatches,010-02648-30,https://support.garmin.com/en-US/?partNumber=0...
3,https://www.garmin.com/en-US/p/819761,819761,Instinct Crossover - Standard Edition,Outdoor Recreation,Adventure Watches,010-02730-13,https://support.garmin.com/en-US/?partNumber=0...
4,https://www.garmin.com/en-US/p/785411,785411,vivomove Trend,Sports & Fitness,Fitness Tracking,010-02665-02,https://support.garmin.com/en-US/?partNumber=0...


In [232]:
df['file_name'] = df['product_name'].str.replace(' ', '_') + "_OM_EN-US.pdf"


In [233]:
df.head()

,product_url,product_id,product_name,product_category,product_subcategory,part_number,product_manual_url,file_name
0,https://www.garmin.com/en-US/p/886725,886725,Forerunner 965,Sports & Fitness,Running,010-02809-00,https://support.garmin.com/en-US/?partNumber=0...,Forerunner_965_OM_EN-US.pdf
1,https://www.garmin.com/en-US/p/714945,714945,Bounce,Sports & Fitness,Just for Kids,010-02448-02,https://support.garmin.com/en-US/?partNumber=0...,Bounce_OM_EN-US.pdf
2,https://www.garmin.com/en-US/p/783467,783467,MARQ Adventurer (Gen 2),Sports & Fitness,Luxury Smartwatches,010-02648-30,https://support.garmin.com/en-US/?partNumber=0...,MARQ_Adventurer_(Gen_2)_OM_EN-US.pdf
3,https://www.garmin.com/en-US/p/819761,819761,Instinct Crossover - Standard Edition,Outdoor Recreation,Adventure Watches,010-02730-13,https://support.garmin.com/en-US/?partNumber=0...,Instinct_Crossover_-_Standard_Edition_OM_EN-US...
4,https://www.garmin.com/en-US/p/785411,785411,vivomove Trend,Sports & Fitness,Fitness Tracking,010-02665-02,https://support.garmin.com/en-US/?partNumber=0...,vivomove_Trend_OM_EN-US.pdf


In [234]:
df.to_csv('current_wearables.csv')